In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
# Building an ANN

In [ ]:
dataset_=pd.read_csv("/content/fmnist_small.csv")


In [ ]:
torch.manual_seed(143)

In [ ]:
dataset_.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X=dataset_.iloc[:,1:].values
y=dataset_.iloc[:,0].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=143)


In [ ]:
# Scaling the feature

X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
# Create customDataset class
# Remember -> pytorch always want to see the features in tensor of datatype float
#             and labels in tensor of dtype long
from torch.utils.data import DataLoader,Dataset

class customDataset(Dataset):

  def __init__(self,X_feature,y_label):
    self.X_feature = torch.tensor(X_feature,dtype=torch.float32)
    self.y_label = torch.tensor(y_label,dtype=torch.long)

  def __len__(self):
    return len(self.X_feature)

  def __getitem__(self,index):
    return self.X_feature[index],self.y_label[index]


In [ ]:
train_dataset=customDataset(X_train,y_train)

In [ ]:
len(train_dataset)

4800

In [ ]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [ ]:
test_dataset=customDataset(X_test,y_test)

In [ ]:
# Loders objects

train_loder = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loder = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
# Pytorch automatically adds lastlayer activation to softmax
class my_model(nn.Module):

  def __init__(self,X_features):

    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(X_features.shape[1],128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10),
    )
  def forward(self,X_features):
    return self.model(X_features)


In [ ]:
epochs=100
learning_rate=0.1

In [ ]:
model=my_model(X_train)

loss_func=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(model.parameters(),learning_rate)



In [ ]:
len(train_loder)

150

In [ ]:
for epoch in range(epochs):
  total_epoch_loss=0
  for feature,label in train_loder:

    y_pred=model(feature)

    loss = loss_func(y_pred,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_epoch_loss+=loss.item()

  avg_loss = total_epoch_loss/len(train_loder)

  print(f'{epoch+1} : Loss:{avg_loss}')


1 : Loss:1.3730025859673818
2 : Loss:0.7792333497603734
3 : Loss:0.6477915241320928
4 : Loss:0.5717420694231987
5 : Loss:0.5257387482126554
6 : Loss:0.4928028933207194
7 : Loss:0.46758899023135503
8 : Loss:0.439824258039395
9 : Loss:0.4191179216901461
10 : Loss:0.3939172958334287
11 : Loss:0.3731004280348619
12 : Loss:0.36449146827061973
13 : Loss:0.34896145343780516
14 : Loss:0.3278542081514994
15 : Loss:0.3311816702783108
16 : Loss:0.31309343439837295
17 : Loss:0.2998190579811732
18 : Loss:0.285160740762949
19 : Loss:0.2927793886264165
20 : Loss:0.2780661736428738
21 : Loss:0.2538643203179042
22 : Loss:0.25232899002730846
23 : Loss:0.2501535420368115
24 : Loss:0.23579682752490044
25 : Loss:0.23215199835598468
26 : Loss:0.2124624769637982
27 : Loss:0.2060824250554045
28 : Loss:0.2061520100136598
29 : Loss:0.20910408886770407
30 : Loss:0.19058555046717326
31 : Loss:0.19004529636353254
32 : Loss:0.17798512438933053
33 : Loss:0.18286396929373344
34 : Loss:0.18869106172273556
35 : Loss:0.

In [ ]:
# Set model to eval mode
model.eval()

my_model(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in test_loder:
    outputs=model(batch_features)

    predicted_labels=torch.max(outputs,1)[1]

    total=total+batch_labels.shape[0]

    correct+=(predicted_labels==batch_labels).sum().item()

print(correct/total)


0.8466666666666667


Training of GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #


In [ ]:
model = my_model(X_train)
model=model.to(device) #

In [ ]:

loss_func=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(model.parameters(),learning_rate)

In [ ]:
for epoch in range(epochs):
  total_epoch_loss=0
  for feature,label in train_loder:

    feature,label=feature.to(device),label.to(device) #

    y_pred=model(feature)

    loss = loss_func(y_pred,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_epoch_loss+=loss.item()

  avg_loss = total_epoch_loss/len(train_loder)

  print(f'{epoch+1} : Loss:{avg_loss}')


1 : Loss:1.3310134307543438
2 : Loss:0.7828498880068461
3 : Loss:0.6523461778958638
4 : Loss:0.5829926305015882
5 : Loss:0.5198263344168663
6 : Loss:0.4930828246474266
7 : Loss:0.466365293264389
8 : Loss:0.43726284672816595
9 : Loss:0.4208666837712129
10 : Loss:0.40228717535734176
11 : Loss:0.3757937490940094
12 : Loss:0.36800823961695034
13 : Loss:0.359536764472723
14 : Loss:0.35058757176001865
15 : Loss:0.32422871291637423
16 : Loss:0.3193486044804255
17 : Loss:0.3012927364806334
18 : Loss:0.2903807316720486
19 : Loss:0.2965108414491018
20 : Loss:0.2713577283422152
21 : Loss:0.2782887872556845
22 : Loss:0.2503578602274259
23 : Loss:0.25149555869400503
24 : Loss:0.24740154442687828
25 : Loss:0.22900102173288664
26 : Loss:0.22057335237661999
27 : Loss:0.23119453025360903
28 : Loss:0.218529580210646
29 : Loss:0.22208120883752902
30 : Loss:0.20523691661655902
31 : Loss:0.19309108937780062
32 : Loss:0.18532425255825122
33 : Loss:0.17247808689872424
34 : Loss:0.1796586647629738
35 : Loss:0

In [ ]:
model.eval()

my_model(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
total=0
correct=0

with torch.no_grad():
  for feutures,labels in test_loder:
    feutures,labels = feutures.to(device),labels.to(device) #

    outputs = model(feutures)

    predicted_label = torch.max(outputs,1)[1]

    total +=feature.shape[0]

    correct += (predicted_label==labels).sum().item()
print("The Acurracy: ",correct/total)
# since i used the same dataset without incresing the data that's why there is no increse in accuracy!!

The Acurracy:  0.8388157894736842


In [ ]:
# We can use Big Number of batch size's in gpu traing

In [ ]:
# Increase accuracy and decrese the cause of overfitting by
# 1) increasing the dataset
# 2) Reducing the Complexity of NN architecture
# 3) using Adm optimizer or differnt optimizer
# 4) differnt lr(learning rate) values
# 5) with multipuple epochs
# 6) BN(batch Normalization)                                              #
# 7) Hyperparameter tuning
# 8) Regularization # By adding the penality term to the loss function.   #
# 9) DropOut layer.                                                       #
# 10) By data agumentation (like tranforming the dataset)
# 11) Early stoping

In [ ]:
# Evaluation with train dataset
total=0
correct=0

with torch.no_grad():
  for feutures,labels in train_loder:
    feutures,labels = feutures.to(device),labels.to(device) #

    outputs = model(feutures)

    predicted_label = torch.max(outputs,1)[1]

    total +=feature.shape[0]

    correct += (predicted_label==labels).sum().item()
print("The Acurracy: ",correct/total)

The Acurracy:  0.99625


In [ ]:
# Therfore we can observe from the above the model is byhating the things->Overfitting
# Which is not good for the model

In [ ]:
# The methods to reduce the overfitting
# Next notebook